In [5]:
import pandas as pd
import numpy as np
from IPython.display import display #This will help displaying two 'returns' in one output cell

# This will ensure that floats in our data show up with 2 decimals and no other format
pd.options.display.float_format = '{:.2f}'.format

In [6]:
df = pd.read_csv('data/raw/data.csv', index_col=0)

# This piece of code is dedicated to clean up the dataframe to make it easer to work with. 

# This will reduce the number of positions and will increase the number of 
# observations for each position type. I.E. unnecessary waste removed
df['Position'] = df['Position'].map({'GK': 'GK', 'CAM': 'CAM','CB': 'CB', 'CDM': 'CDM',
                                      'CM': 'CM','LB': 'LB','LM': 'LM','LW': 'LW',
                                      'RB': 'RB','RM': 'RM','RW': 'RW','ST': 'ST',
                                      'CF': 'ST','LAM': 'CAM','LCB': 'CB','LCM': 'CM', 
                                      'LDM': 'CDM','LF': 'LW','LS': 'ST', 'LWB': 'LB', 
                                      'RAM': 'CAM', 'RCB': 'CB','RCM': 'CM', 'RDM': 'CDM', 
                                      'RF': 'RW', 'RS': 'ST','RWB': 'RB'})


# Function that turn a string variable "€100M" to 100,000,000   
def money_string_replace(variable_name):
    df[variable_name] = (df[variable_name].str[1:].replace(r'[KM]+$', '', regex=True).astype(float) * 
                 df[variable_name].str.extract(r'[\d\.]+([KM]+)', expand=False)
                 .fillna(1)
                 .replace(['K','M'], [10**3, 10**6]).astype(int))

money_string_replace('Value')
money_string_replace('Wage')
money_string_replace('Release Clause')

# Creating a dummy variable to identify if a player is left footed or not
df.loc[df['Preferred Foot'] == "Left", 'Preferred Foot = Left'] = 1
df.loc[df['Preferred Foot'] == "Right", 'Preferred Foot = Left'] = 0

# Creating a new column to use in regression for a more accurate prediction of player growth. 
# Potential - Growth
df['improvement'] = df['Potential'] - df['Overall']


# Splitting the work rate column into attacking and defensive rates. 
df['Work Rates'] = df['Work Rate'].str.split('/')

# Allocating each list item into the according work type
df['Attacking Rate'] = df['Work Rates'].str.get(0)
# Removing any whitespace
df['Attacking Rate'] = df['Attacking Rate'].str.strip()
# Creating a numeric variable for each work type
df.loc[df['Attacking Rate'] == "Low", 'Attacking Rate'] = 1
df.loc[df['Attacking Rate'] == "Medium", 'Attacking Rate'] = 2
df.loc[df['Attacking Rate'] == "High", 'Attacking Rate'] = 3

df['Defensive Rate'] = df['Work Rates'].str.get(1)
df['Defensive Rate'] = df['Defensive Rate'].str.strip()
df.loc[df['Defensive Rate'] == "Low", 'Defensive Rate'] = 1
df.loc[df['Defensive Rate'] == "Medium", 'Defensive Rate'] = 2
df.loc[df['Defensive Rate'] == "High", 'Defensive Rate'] = 3

# Converting pounds to kg
df["Weight"] = df["Weight"].str[:3].astype(float) /2.205

# Converting Feet and Inches string to a CM value
# Splitting Feet from Inches
df['HeightCM'] = df['Height'].str.split("'")
# Further split of feet and inches
df['feet'] = df['HeightCM'].str.get(0).astype(float)
df['inch'] = df['HeightCM'].str.get(1).astype(float)
# Replcaing Old string with new CM value
df['Height'] = (df['feet']*12+df['inch'])*2.54

df.loc[df['Body Type'] == "Lean", 'Body Type'] = 1
df.loc[df['Body Type'] == "Normal", 'Body Type'] = 2
df.loc[df['Body Type'] == "Stocky", 'Body Type'] = 3
df.loc[df['Body Type'] == "Messi", 'Body Type'] = 2
df.loc[df['Body Type'] == "C. Ronaldo", 'Body Type'] = 2
df.loc[df['Body Type'] == "Neymar", 'Body Type'] = 1
df.loc[df['Body Type'] == "Courtois", 'Body Type'] = 2
df.loc[df['Body Type'] == "Shaqiri", 'Body Type'] = 2
df.loc[df['Body Type'] == "Akinfenwa", 'Body Type'] = 3
df.loc[df['Body Type'] == "PLAYER_BODY_TYPE_25", 'Body Type'] = np.nan

# Remove rows with Position = NaN
df.dropna(subset=['Position'], inplace=True)

# Dropping unneccesary columns
df.drop(["Photo","Flag","Club Logo","Special","ID",'Preferred Foot','Real Face',
        'Jersey Number','Loaned From','LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LM',
         'LB','LCB','CB','RCB','RB','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB','Work Rate','Work Rates',
         'HeightCM', 'feet', 'inch'], axis=1, inplace=True)

combined_columns_data = df

combined_columns_data['control'] = combined_columns_data[['Dribbling', 'BallControl']].mean(axis=1)
combined_columns_data['defending'] = combined_columns_data[['Interceptions', 'Marking','StandingTackle','SlidingTackle']].mean(axis=1)
combined_columns_data['gk'] = combined_columns_data[['GKDiving', 'GKHandling','GKKicking','GKPositioning','GKReflexes']].mean(axis=1)
combined_columns_data['passing'] = combined_columns_data[['Crossing', 'ShortPassing','Volleys','LongPassing']].mean(axis=1)
combined_columns_data['physical'] = combined_columns_data[['Agility', 'Reactions','Balance','Jumping','Stamina','Strength','Composure']].mean(axis=1)
combined_columns_data['shooting'] = combined_columns_data[['Finishing', 'Curve','FKAccuracy','ShotPower', 'LongShots','Penalties']].mean(axis=1)
combined_columns_data['skill'] = combined_columns_data[['Positioning', 'Vision']].mean(axis=1)
combined_columns_data['speed'] = combined_columns_data[['Acceleration', 'SprintSpeed']].mean(axis=1)
combined_columns_data['heading'] = combined_columns_data[['HeadingAccuracy']].mean(axis=1)

combined_columns_data.drop(['Dribbling', 'BallControl','Interceptions', 'Marking',
                             'StandingTackle','SlidingTackle','GKDiving', 'GKHandling',
                             'GKKicking','GKPositioning','GKReflexes','Crossing',
                             'ShortPassing','Volleys','LongPassing','Agility',
                             'Reactions','Balance','Jumping','Stamina',
                             'Strength','Composure','Finishing', 'Curve',
                             'FKAccuracy','ShotPower', 'LongShots','Penalties',
                             'Positioning', 'Vision','Acceleration', 'SprintSpeed','HeadingAccuracy'], axis=1, inplace=True)

# Make all column names lower case for future ease of use
combined_columns_data.columns = map(str.lower, combined_columns_data.columns)   

combined_columns_data.to_csv('data/processed/reduced_clean_data.csv')

In [7]:
combined_columns_data.position.unique()

array(['RW', 'ST', 'LW', 'GK', 'CM', 'CB', 'CDM', 'CAM', 'RM', 'LM', 'LB',
       'RB'], dtype=object)